# DATA VALIDATIONS - GREAT EXPECTATIONS

================================================


Milestone 3 - Data Validation with Great Expectations (GX)


Name  : Aulia Putri Nugrahani              

              
Batch : FTDS-029-HCK

In this notebook, we will perform data validation tests using Python's Great Expectations (GX) library.  
This step helps ensure that the cleaned dataset meets the expected data quality standards before moving on to analysis or modeling.
We will use the cleaned dataset that was prepared earlier in this project.

Dataset path: airflow/data/P2M3_Aulia-Putri_data_cleaned.csv

================================================



In [2]:
# Installing Great Expectations
!pip install -q "great-expectations==0.18.19"

In [ ]:
# import libraries
from great_expectations.data_context import FileDataContext

# 1. Creating Data Context

A Data Context is the main configuration hub in a Great Expectations (GX) project. It manages datasources, expectations, validations, and stores all project settings.

This step initializes the context so we can start building and running data validations.

In [4]:
# Create a data context

context = FileDataContext.create(project_root_dir='./')

# 2. Connecting to Data Source

In [5]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'adidas-sales-data'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'adidas-sales'
path_to_data = 'P2M3_Aulia-Putri_data_cleaned.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# 3. Create an Expectation Suite 

In [6]:
# Creat an expectation suite
expectation_suite_name = 'expectation-adidas-sales-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,retailer,retailer_id,invoice_date,region,state,city,product,price_per_unit,units_sold,total_sales,operating_profit,operating_margin,sales_method
0,Foot Locker,1185732,2020-01-01,Northeast,New York,New York,Men's Street Footwear,50.0,1200,600000.0,300000.0,0.50,In-store
1,Foot Locker,1185732,2020-01-02,Northeast,New York,New York,Men's Athletic Footwear,50.0,1000,500000.0,150000.0,0.30,In-store
2,Foot Locker,1185732,2020-01-04,Northeast,New York,New York,Women's Athletic Footwear,45.0,850,382500.0,133875.0,0.35,In-store
3,Foot Locker,1185732,2020-01-05,Northeast,New York,New York,Men's Apparel,60.0,900,540000.0,162000.0,0.30,In-store
4,Foot Locker,1185732,2020-01-06,Northeast,New York,New York,Women's Apparel,50.0,1000,500000.0,125000.0,0.25,In-store


# 4.Expectations

## 4.1. Expectations 1: To Be Unique

In this dataset, no single column has completely unique values for each row. To solve this, we created a new column called invoice_key. This key is built by combining several pieces of information:

1. The first letter of the retailer name

2. The retailer ID

3. A short code for the city

4. The invoice date (without dashes)

5. A code representing the product

6. The first letter of the sales method

7. The operating margin (converted to 3 digits without the dot)

By putting all of these together, we aim to create a unique identifier for each transaction in the dataset.


### A. Adding Invoice Key Column

To perform data validation, we first need to access the actual data. Great Expectations stores the data in what's called a "batch." Here, we're extracting the active batch from the validator and converting it into a regular Pandas DataFrame so we can create a new column.

In [7]:
# Extract the dataframe from validator
df = validator.active_batch.data.dataframe


#### Step 1: Creating State Code

For the next step, we're going to create a dictionary that maps full U.S. state names to their corresponding standard two-letter abbreviations (state codes). These abbreviations are widely used in datasets to ensure consistency and reduce redundancy.

We then use this mapping to generate a new column called state_code, which will be used later in constructing a unique and standardized invoice_key.

The list of state abbreviations follows the official format published by the Federal Aviation Administration (FAA), available at: https://www.faa.gov/air_traffic/publications/atpubs/cnt_html/appendix_a.html

In [8]:
# Create State Code Map
state_code_map = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}


After defining the state code mapping, we apply it to the state column using the .map() function. This creates a new column called state_code, which stores the two-letter abbreviation for each U.S. state. This standardization helps keep our data consistent and easier to work with for downstream processing.

In [9]:
# Map the state column to create a new state_code column
df["state_code"] = df["state"].map(state_code_map)

#### Step 2: Creating Product Code Mapping

To simplify and standardize the product column, we map each product category to a shortened code using a dictionary. This makes our data more compact and helps when generating unique identifiers such as the invoice_key.

In [10]:
# Define product code mapping
product_code_map = {
    "Men's Street Footwear": "M-SF",
    "Men's Athletic Footwear": "M-AF",
    "Women's Athletic Footwear": "W-AF",
    "Women's Street Footwear": "W-SF",
    "Men's Apparel": "M-AP",
    "Women's Apparel": "W-AP"
}

Each product name is abbreviated using the format [Gender Initial]-[Category Abbreviation] to keep it readable but concise.

#### Step 3: Creating Invoice Key

To uniquely identify each transaction, we create a new column called invoice_key. This key is constructed by combining multiple relevant fields from the dataset to ensure uniqueness and traceability.

In [11]:
df["invoice_key"] = (
    df["retailer"].str[:1].str.upper() +                    # First letter of retailer name
    df["retailer_id"].astype(str) + "-" +                   # Retailer ID
    df["state_code"] + "-" +                                # Mapped state code                                 
    df["invoice_date"].str.replace("-", "") + "-" +         # Invoice date (formatted as YYYYMMDD)
    df["product"].map(product_code_map) + "-" +             # Mapped product code
    df["sales_method"].str[0].str.upper() +                 # First letter of sales method
    df["operating_margin"].astype(str).str.replace(".", "", regex=False).str.zfill(3)
)

Sometimes, the invoice_key column may have duplicate values. To ensure each invoice key is unique, we group the data by invoice_key and add a count suffix (_0, _1, _2,) to duplicates using the cumcount() function. This way, each row has a distinct invoice key for accurate identification.

In [12]:
df["invoice_key"] = (
    df["invoice_key"] + "_" +
    df.groupby("invoice_key").cumcount().astype(str)
)

#### Step 4: Verifying Unique Value

To verify the data, we print the total number of rows using df.shape[0], showing how many records are in the dataset. Then, we count the number of unique invoice keys with df['invoice_key'].nunique(), helping us confirm the uniqueness and completeness of the invoice keys.

In [13]:
print(f"Total number of rows: {df.shape[0]}")
print(f"Number of unique invoice_key: {df['invoice_key'].nunique()}")


Total number of rows: 9647
Number of unique invoice_key: 9647


To verify the invoice keys, we display the first 10 values of the invoice_key column using df['invoice_key'].head(10). This helps us quickly check the format and correctness of the generated keys.

In [14]:
# Checking 10 invoice_key
df['invoice_key'].head(10)

0    F1185732-NY-20200101-M-SF-I005_0
1    F1185732-NY-20200102-M-AF-I003_0
2    F1185732-NY-20200104-W-AF-I035_0
3    F1185732-NY-20200105-M-AP-I003_0
4    F1185732-NY-20200106-W-AP-I025_0
5    F1185732-NY-20200107-M-SF-I005_0
6    F1185732-NY-20200108-M-AF-O003_0
7    F1185732-NY-20200121-W-SF-O035_0
8    F1185732-NY-20200122-W-AF-O035_0
9    F1185732-NY-20200123-M-AP-O003_0
Name: invoice_key, dtype: object

### B. Expectation: To Be Unique

This validation ensures that each row in the dataset has a unique identifier. Since invoice_key is a combination of multiple columns intended to uniquely identify transactions, we expect no duplicate values in this column

In [15]:
# Expectation : Column `invoice_key` must be unique
validator.expect_column_values_to_be_unique('invoice_key')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9647,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The result confirms that the invoice_key column passed the uniqueness check successfully. Out of 9,647 rows, there were no duplicates or missing values, and 100% of the values met the expectation. This means our generated invoice_key is a valid unique identifier for each record in the dataset.

## 4.2. Expectation 2: To Be Between min_value and max_value

To ensure that the operating_margin column contains valid values, we apply an expectation that all entries must fall between 0 and 1. A company's operating margin represents how much it earns on every dollar of sales after deducting variable costs but before accounting for interest or taxes. The values are already in decimal form (e.g., 0.25 = 25%), which is a common format in financial analysis. This expectation ensures that the data stays within a realistic and interpretable range for operating profitability.

Source: https://www.investopedia.com/terms/n/net_margin.asp#:~:text=Most%20often%2C%20the%20net%20profit,multiplying%20that%20figure%20by%20100.

In [17]:
# Expectation : Column `operating_margin` must be less than 1
validator.expect_column_values_to_be_between(
    column='operating_margin',
    min_value=0,
    max_value=1
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9647,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

This result confirms that the operating_margin column successfully passed the validation check. All 9,647 values fall between 0 and 1, with no missing or unexpected values. The "success": true flag indicates that the data aligns perfectly with the defined expectation, and there were no errors or exceptions during the check.

## 4.3. Expectation 3: To be in set

This expectation checks whether every value in the region column belongs to a predefined list of allowed values: ['Northeast', 'South', 'West', 'Midwest', 'Southeast']. It ensures that the data is consistent and that there are no unexpected or misspelled region names. This type of validation is especially useful for categorical data where only a specific set of values should appea

In [18]:
validator.expect_column_values_to_be_in_set(
    column='region',
    value_set=['Northeast', 'South', 'West', 'Midwest', 'Southeast']
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9647,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation result shows that **all 9,647 entries** in the `region` column match the expected set of values. There were **no unexpected or missing values**, which confirms that the data is clean and consistent with the defined categories (Northeast, South, West, Midwest, Southeast). This indicates the column meets the expectation successfully without errors or exceptions.


## 4.4. Expectations 4: To be in type

In this expectation, we check whether all values in the price_per_unit column are of a specific data type, which is float64. This ensures that the column contains consistent numerical values, which is important for calculations like pricing analysis or revenue aggregation. By validating the data type, we reduce the risk of errors due to incorrect or mixed data types in downstream processes such as modeling or reporting.

In [19]:
validator.expect_column_values_to_be_of_type(
    column='price_per_unit',
    type_='float64'
)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The validation result confirms that the expectation was successful: all values in the `price_per_unit` column are of the expected data type `float64`. The `"observed_value"` key indicates the actual data type found, which matches our expectation. No exceptions or errors occurred during the check, ensuring data consistency in this numeric column.

## 4.5. Expectation 5: Expect Column Values to Not Match 

The product column is expected to contain descriptive names of product categories, such as "Men's Apparel" or "Women's Street Footwear." Since product names should consist only of text, this expectation ensures that there are no numerical digits present in the values. The presence of digits might indicate data entry errors or inconsistent formatting, so this validation helps maintain the cleanliness and integrity of the product information.

In [20]:
validator.expect_column_values_to_not_match_regex(
    column='product',
    regex=r'\d'
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9647,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

The expectation was successfully validated across all 9,647 rows. There were no unexpected values, meaning all entries passed the defined rule. Additionally, there were no missing values, and the data is complete and clean in accordance with the applied expectation. This confirms the data quality for the tested column is high.

## 4.6.Expectations 6: Expect Table Column Count to Equal

In this expectation, we check whether the total number of columns in our dataset is exactly 15. Originally, the dataset contained 13 columns. During preprocessing, we added two new columns: state_code (mapped from the state column) and invoice_key (a unique identifier composed of multiple column values). This brings the total expected column count to 15.

By using:

In [21]:
validator.expect_table_column_count_to_equal(value=15)


Calculating Metrics:   0%|          | 0/3 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 15
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

we ensure that no unexpected columns were added or missing during the preprocessing steps. This helps maintain the integrity and consistency of the dataset schema throughout the data pipeline.

## 4.7 Expectations 7: Expect Column Value Lengths to Equal

In this expectation, we validate that every value in the retailer_id column consists of exactly 7 characters. This ensures that the retailer IDs follow a consistent format, which is important for data integrity, especially when these IDs are used for merging, filtering, or generating unique identifiers like invoice_key.


In [22]:
validator.expect_column_value_lengths_to_equal(
    column="retailer_id",
    value=7
)


Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 9647,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In this expectation, we confirmed that all 9,647 values in the retailer_id column meet the requirement of having exactly 7 characters. There are no missing or unexpected values, indicating the data is clean and consistently formatted for reliable downstream use.

# 5. Save Expectation Suite

ChatGPT said:
In this step, we save the current set of expectations into an Expectation Suite. By setting discard_failed_expectations=False, we ensure that all expectations are saved, including those that may have failed during validation. This allows for consistent tracking, debugging, and reuse of expectations across different datasets or pipeline runs.

In [23]:
# Save into Expectation Suite

validator.save_expectation_suite(discard_failed_expectations=False)

# 6. Checkpoint

In this step, we create a checkpoint named 'checkpoint_1' to bundle the Expectation Suite with the data context and validator. This checkpoint allows us to run validations in a structured and reusable way, making it easier to monitor data quality across pipeline executions.

In [24]:
# Create a checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)


In this step, we run the validate() method to execute all expectations in the suite against the current dataset. The result is stored in validation_result, which contains detailed information about whether the data meets the defined expectations, including success status, counts of unexpected or missing values, and any errors encountered during validation.

In [25]:
validation_result = validator.validate()

Calculating Metrics:   0%|          | 0/30 [00:00<?, ?it/s]

The value of validation_result["success"] shows whether the data passed the checks:

- True means all checks passed.

- False means at least one check failed.

This helps quickly show if the data follows the expected rules.

In [26]:
validation_result["success"]

True

All data checks passed successfully. The dataset meets all the rules or expectations that were defined.

Lastly, we're going to create a report (in HTML format) that shows the results of the data checks. The report makes it easier to see which checks passed and which ones failed.

In [27]:
# Build data docs

context.build_data_docs()

{'local_site': 'file:///Users/nugrahani/p2-ftds029-hck-m3-aulianugrahani/gx/uncommitted/data_docs/local_site/index.html'}